## Scraping MLS Player Stats

Scraping www.mlssoccer.com/ for player statistics

### Import Necessary Modules

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime 
from dateutil import parser
import os
import time
import itertools
import re
import pickle

### Scrape Statistics

#### Define Scraper and Helpers

In [3]:
def send_request(url, params={}, attempts=3):

    cnt = 0
    while cnt <= attempts:
        cnt += 1        
        # Send Request
        response = requests.get(url, params)
        # Check Status
        print(response.url, '\n', response.status_code, response.reason)
        if (response.status_code >= 200) and (response.status_code < 300):           
            return response
        # If bad status, pause before trying again
        print('Pause, then retry')
        time.sleep(15)
    return None

In [4]:
def map_id_to_club(franchise_id):
    
    franchise_map = {11091: 'ATL',
                     1207: 'CHI',
                     436: 'COL',
                     454: 'CLB',
                     1326: 'DC',
                     1903: 'DAL',
                     1897: 'HOU',
                     1230: 'LA',
                     11690: 'LAFC',
                     6977: 'MNUFC',
                     1616: 'MTL',
                     928: 'NE',
                     9668: 'NYCFC',
                     399: 'NYRB',
                     6900: 'ORL',
                     5513: 'PHI',
                     1581: 'POR',
                     1899: 'RSL',
                     1131: 'SJ',
                     3500: 'SEA',
                     421: 'KC',
                     2077: 'TOR',
                     1708: 'VAN'}
    return franchise_map[franchise_id]
        

In [5]:
def check_for_data(stats_table):
    # Check if the table has any 
    odd_rows = stats_table.findAll('tr', {'class': 'odd'})
    if odd_rows[0].text.strip() == 'Stats Unavailable':
        return False
    return True    

In [6]:
def split_player_name(df):
    first = []
    last = []
    for t in df.Player.str.split():
        if len(t) == 1:
            first.append(t[0])
            last.append('')
        elif len(t) == 2:
            first.append(t[0])
            last.append(t[1])
        else:
            first.append(t[0])
            last.append(' '.join(t[1:]))
    
    df['Last'] = last
    df['First'] = first  
    
    df = df.drop('Player', axis=1)
    
    return df
        

In [7]:
# # Find and parse franchise popup menu to get list of teams
# soup = BeautifulSoup(response.text, 'lxml')
# franchise_select = soup.find('select', {'id': 'edit-franchise', 'name': 'franchise'})
# franchise = []
# # start with second index since first is generic "Select A Club"
# for team in franchise_select.findAll('option')[1:]:
#     franchise.append(int(team['value']))

In [8]:
def generate_param_combos(param_groups):
    params = []
    for combo in itertools.product(*param_groups):
        params.append({'franchise': combo[0],
                       'group': combo[1],
                       'season_type': combo[2],
                       'year': combo[3],
                       'page': combo[4]})
    return params

In [9]:
def scrape_player_stats(base_url, params, return_last_pg=False):

    response = send_request(base_url, params)
    if response is None:
        return None
    
    # Parse HTML
    soup = BeautifulSoup(response.text,'lxml')
    
    stats_table = soup.find('table')
    
    if not check_for_data(stats_table):
        return None
    
    stats_df = extract_stats(stats_table)
    # Add Year Column
    stats_df['Year'] = np.repeat(int(params['year']), len(stats_df))
    # Add Season Column
    stats_df['Season'] = np.repeat(params['season_type'], len(stats_df))
    
    if return_last_pg:
        return stats_df, get_last_page(soup)
    
    return stats_df     

In [10]:
def extract_stats(stat_table):
    
    # Extract salary data
    stat_header = []
    stat_data = []
    for row in stat_table.findAll('tr'):
        row_data = []

        # Get row type and check if header or data row
        row_type = row.findChild().name
        if row_type == 'th':
            # Extract header
            for h in stat_table.findAll('th'):
                stat_header.append(h.text) 
        else:
            # Extract data
            for data in row.findAll('td'):
                row_data.append(data.text)
            stat_data.append(row_data)    
            
    # Compile stat dataframe
    stat_df = pd.DataFrame(stat_data, columns=stat_header)
    
    # Strip any whitespace from column names
    stat_df.columns = stat_df.columns.str.strip()
    
    # Replace blanks with nans
    stat_df = stat_df.replace('', np.nan)
        
    return stat_df

In [11]:
def get_last_page(soup):

    last_pg_url = soup.find('li', {'class': 'pager-last last'}).find('a')['href']
    last_pg = int(re.search('(?<=page=)\d+', last_pg_url).group())
    
    return last_pg

In [12]:
def scrape_group_stats(base_url, group, year=np.arange(2007, 2018), franchise='select', season_type='REG'):
    
    group_df = pd.DataFrame()
    combos = generate_param_combos([[franchise], [group], [season_type], year, [0]])
    for params in combos: 
        stats_df, last_pg = scrape_player_stats(base_url, params, True)
        
        if stats_df is None:
            continue
            
        # Scrape first page of results
        group_df = pd.concat([group_df, stats_df], axis=0)
        # Add pause to prevent 429 status
        print(datetime.now())
        time.sleep(np.random.uniform(2, 5)) 
        
        # Scrape next through last
        for idx in range(1, last_pg+1):
            params['page'] = idx
            stats_df = scrape_player_stats(base_url, params)
            if stats_df is None:
                continue
            group_df = pd.concat([group_df, stats_df], axis=0)            
            # Add pause to prevent 429 status
            print(datetime.now())
            time.sleep(np.random.uniform(2, 5)) 
            
    # Reset index to remove duplicates created during concatenation
    group_df.reset_index(inplace=True, drop=True)
    
    return group_df        

#### Send Request to Base URL and Verify Site is Up

In [1]:
import requests
base_url = 'http://www.mlssoccer.com/stats/season'
response = requests.get(base_url)
# response.status_code, response.url

In [2]:
from bs4 import BeautifulSoup

In [4]:
response.text

'<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->\n<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->\n<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->\n<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->\n<!--[if IE 9]><html class="eq-ie9"  lang="en" dir="ltr"><![endif]-->\n<!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html class="no-ie"  lang="en" dir="ltr"\n  xmlns:og="http://ogp.me/ns#"\n  xmlns:article="http://ogp.me/ns/article#"\n  xmlns:book="http://ogp.me/ns/book#"\n  xmlns:profile="http://ogp.me/ns/profile#"\n  xmlns:video="http://ogp.me/ns/video#"><!--<![endif]-->\n\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n\n  <meta charset="utf-8" />\n<link rel="apple-touch-icon" sizes="144x144" href="/sites/league/themes/league/img/apple-touch-icon-144x144-precomposed.png" />\n<link rel="apple-touch-icon" sizes="152x152" href="/sites/league/themes/l

#### Define parameter options

In [14]:
# group = ['goals', 'assists', 'shots', 'fouls', 'goalkeeping']
# for grp in group:
#     print('Scraping:', grp, datetime.now(), '\n')
#     df = scrape_group_stats(base_url, grp, np.arange(2007, 2018, 1))
#     df.to_pickle(grp + '_df.pkl')
#     print('Completed Scraping:', grp)

In [15]:
fouls_df = scrape_group_stats(base_url, 'fouls')

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=0 
 200 OK
2018-01-26 17:36:43.348369
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=1 
 200 OK
2018-01-26 17:36:53.232375
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=2 
 200 OK
2018-01-26 17:37:04.300632
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=3 
 200 OK
2018-01-26 17:37:18.309698
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=4 
 200 OK
2018-01-26 17:37:30.313162
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=5 
 200 OK
2018-01-26 17:37:38.724534
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=6 
 200 OK
2018-01-26 17:37:45.731797
https://www.mlssoccer.com/stats/season?fr

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=13 
 200 OK
2018-01-26 17:45:10.260328
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=14 
 200 OK
2018-01-26 17:45:16.394818
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=15 
 200 OK
2018-01-26 17:45:23.879019
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=16 
 200 OK
2018-01-26 17:45:33.090253
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=0 
 200 OK
2018-01-26 17:45:39.355003
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=1 
 200 OK
2018-01-26 17:45:47.372754
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=2 
 200 OK
2018-01-26 17:45:55.433108
https://www.mlssoccer.com/stats/seaso

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=14 
 200 OK
2018-01-26 17:55:06.438992
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=15 
 200 OK
2018-01-26 17:55:18.458530
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=16 
 200 OK
2018-01-26 17:55:28.057691
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=17 
 200 OK
2018-01-26 17:55:44.132065
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=18 
 200 OK
2018-01-26 17:55:54.680042
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=19 
 200 OK
2018-01-26 17:56:06.061365
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=20 
 200 OK
2018-01-26 17:56:18.816256
https://www.mlssoccer.com/stats/se

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=8 
 200 OK
2018-01-26 18:04:58.745835
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=9 
 200 OK
2018-01-26 18:05:08.883285
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=10 
 200 OK
2018-01-26 18:05:15.896533
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=11 
 200 OK
2018-01-26 18:05:24.135915
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=12 
 200 OK
2018-01-26 18:05:30.177516
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=13 
 200 OK
2018-01-26 18:05:37.388959
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=14 
 200 OK
2018-01-26 18:05:47.682119
https://www.mlssoccer.com/stats/seas

In [19]:
fouls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5316 entries, 0 to 5315
Data columns (total 17 columns):
Player    5316 non-null object
Club      5312 non-null object
POS       5316 non-null object
GP        5312 non-null object
GS        5312 non-null object
MINS      5312 non-null object
G         5312 non-null object
A         5312 non-null object
SHTS      5312 non-null object
SOG       5312 non-null object
FC        5312 non-null object
FS        5312 non-null object
OFF       5312 non-null object
YC        5312 non-null object
RC        5312 non-null object
Year      5316 non-null int64
Season    5316 non-null object
dtypes: int64(1), object(16)
memory usage: 706.1+ KB


In [20]:
fouls_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,FC,FS,OFF,YC,RC,Year,Season
0,Juan Toja,DAL,M,27,27,2388,6,1,33,16,83,41,6,4,0,2007,REG
1,Eddie Robinson,HOU,D,25,25,2187,2,2,10,5,70,64,0,11,1,2007,REG
2,Maurice Edu,TOR,M,25,25,2180,4,1,24,8,65,50,1,6,1,2007,REG
3,Jeff Larentowicz,ATL,M,28,28,2455,3,4,28,9,61,30,0,5,0,2007,REG
4,Ryan Cochrane,HOU,D,25,25,2059,1,0,6,1,55,13,0,6,1,2007,REG
5,Shalrie Joseph,NE,M,27,27,2366,4,5,42,14,55,52,5,4,1,2007,REG
6,Alejandro Moreno,CLB,F,29,27,2357,7,7,54,32,54,93,32,3,0,2007,REG
7,Dane Richards,NY,M,28,27,2335,2,6,24,14,50,35,12,5,0,2007,REG
8,Carl Robinson,TOR,M,26,26,2340,2,4,14,8,50,24,0,6,0,2007,REG
9,Ben Olsen,DC,M,24,21,1881,7,7,29,17,49,41,11,8,0,2007,REG


In [21]:
fouls_df.to_pickle('fouls_df.pkl')

### Sandbox